<a href="https://colab.research.google.com/github/aimtyaem/EOInfo/blob/main/Carbon_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd

class CarbonFootprintAnalyzer:
    def __init__(self, energy_costs_file, grid_emissions_file, aws_coefficients_file):
        self.energy_costs = pd.read_csv(energy_costs_file)
        self.grid_factors = pd.read_csv(grid_emissions_file)
        self.aws_coefficients = pd.read_csv(aws_coefficients_file)

    def classify_data(self):
        """Classify the different CSV datasets into unified structure."""
        # Clean and transform energy costs data
        self.energy_costs['cost_per_unit'] = self.energy_costs['cost_per_unit'].str.replace('$', '').astype(float)

        # Classify AWS coefficients
        aws_architectures = self.aws_coefficients[['Architecture', 'Min Watts', 'Max Watts']].copy()
        aws_architectures['Watts Range'] = aws_architectures.apply(
            lambda x: f"{x['Min Watts']:.2f}-{x['Max Watts']:.2f} W", axis=1
        )

        # Classify grid emission factors
        grid_factors_clean = self.grid_factors[['Region', 'emissions_tCO2e', 'energy_source']].copy()
        grid_factors_clean['Type'] = 'Cloud Infrastructure'

        return {
            'energy_costs': self.energy_costs,
            'aws_architectures': aws_architectures,
            'grid_emissions': grid_factors_clean
        }

    def generate_report(self, business_name, aws_usage=None):
        """Generate markdown report with classified data."""
        classified = self.classify_data()

        md = f"# Carbon Footprint Report for {business_name}\n\n"

        # Energy Costs Section
        md += "## Energy Cost Analysis\n"
        md += "| Energy Source | Cost per Unit |\n|---------------|-------------|\n"
        for _, row in classified['energy_costs'].iterrows():
            md += f"| {row['energy_source']} | ${row['cost_per_unit']:.2f} |\n"

        # AWS Emissions Section
        md += "\n## Cloud Infrastructure Profile\n"
        md += "### AWS Region Emissions Factors\n"
        md += "| Region | Emissions Factor (tCO2e) | Data Source |\n|--------|--------------------------|-------------|\n"
        for _, row in classified['grid_emissions'].iterrows():
            md += f"| {row['Region']} | {row['emissions_tCO2e']:.6f} | {row['energy_source']} |\n"

        # Hardware Efficiency Section
        md += "\n### Server Architecture Efficiency\n"
        md += "| Architecture | Power Consumption Range |\n|--------------|--------------------------|\n"
        for _, row in classified['aws_architectures'].iterrows():
            md += f"| {row['Architecture']} | {row['Watts Range']} |\n"

        # Recommendations Section
        md += "\n## Optimization Recommendations\n"
        md += ("1. **Region Optimization**: Consider shifting workloads to lower-emission regions like eu-north-1\n"
               "2. **Architecture Upgrade**: Migrate to Graviton-based instances for better energy efficiency\n"
               "3. **Renewable Energy**: Explore AWS Renewable Energy Programs for carbon offset\n"
               "4. **Instance Right-Sizing**: Use compute-optimized architectures for energy-intensive workloads")

        return md


# Usage Example
analyzer = CarbonFootprintAnalyzer(
    energy_costs_file='/content/drive/MyDrive/Colab Notebooks/energy_costs.csv',
    grid_emissions_file='/content/drive/MyDrive/Colab Notebooks/grid-emissions-factors-aws.csv',
    aws_coefficients_file='/content/drive/MyDrive/Colab Notebooks/coefficients-aws-use.csv'
)

report = analyzer.generate_report(
    business_name="CloudTech Solutions",
    aws_usage={'region': 'us-east-1', 'hours': 8760}
)

with open("cloud_carbon_report.md", "w") as f:
    f.write(report)